In [251]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score

In [252]:
filenames = glob.glob('Datathon*.csv')

In [279]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [280]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')

In [264]:
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [5]:
submit = pd.read_csv('sample_submission_bet_size.csv')
submit.columns = ['ACCOUNT_ID', 'Prediction']

In [6]:
pre_dat = pre_dat.loc[(pre_dat.ACCOUNT_ID.isin(submit.ACCOUNT_ID)).values]

In [281]:
pre_dat = pre_dat.loc[(pre_dat.STATUS_ID == 'S').values]

In [266]:
pre_dat.correct_profit_loss.sum()

2886479.6750997272

In [269]:
pre_dat.loc[(pre_dat.EVENT_ID == 101092958).values, 'PROFIT_LOSS'].sum()

1802972.7825382631

In [267]:
peek_dat.EVENT_ID.value_counts()

101092958    23691
dtype: int64

In [256]:
submit.loc[~submit.ACCOUNT_ID.isin(pre_dat.ACCOUNT_ID), 'ACCOUNT_ID'].nunique()

1001

In [8]:
pre_dat_by_account_event = pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [9]:
win_loss = pre_dat_by_account_event['correct_profit_loss'].sum()

In [15]:
win_loss1 = win_loss > 0

In [16]:
maps = {True: 1, False: -1.5}

In [17]:
win_loss1 = win_loss1.map(maps)

In [18]:
win_loss1.head()

ACCOUNT_ID  EVENT_ID 
1000002     101093076   -1.5
            101093312   -1.5
            101128387    1.0
            101149398   -1.5
            101149516   -1.5
Name: correct_profit_loss, dtype: float64

In [20]:
win_loss1_by_account = win_loss1.groupby(level=0)

In [21]:
win_loss2 = win_loss1_by_account.sum()

In [22]:
win_loss2 = win_loss2.loc[submit.ACCOUNT_ID.tolist()]

In [23]:
win_loss2.head()

ACCOUNT_ID
1004240    -6.5
1009742    10.0
1003908     2.0
1020888    -1.5
1010074    -1.5
Name: correct_profit_loss, dtype: float64

In [24]:
win_loss2.describe()

count    6373.000000
mean       -2.722266
std         9.604065
min       -44.500000
25%        -7.500000
50%        -1.500000
75%         2.000000
max        43.000000
Name: correct_profit_loss, dtype: float64

In [25]:
win_loss2 = win_loss2.reset_index()

In [26]:
win_loss2.columns = ['Account_ID', 'Prediction']

In [27]:
win_loss2 = win_loss2.fillna(-10)

In [28]:
win_loss2.to_csv('win1_loss1_5_fill_minus10.csv', index=False)

In [153]:
win_loss2.head()

,Account_ID,Prediction
0,1004240,-1
1,1009742,12
2,1003908,3
3,1020888,-1
4,1010074,-1


In [154]:
win_loss2.shape

(7374, 2)

In [155]:
win_loss2.isnull().any()

Account_ID    False
Prediction    False
dtype: bool

In [152]:
four_team_pre_dat = pre_dat.loc[(pre_dat.MATCH.str.contains('Australia') |
                                 pre_dat.MATCH.str.contains('New Zealand') |
                                 pre_dat.MATCH.str.contains('India') |
                                 pre_dat.MATCH.str.contains('South Africa')).values]

In [153]:
four_team_pre_dat_by_account_event = four_team_pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [154]:
win_loss = four_team_pre_dat_by_account_event['correct_profit_loss'].sum()

In [237]:
win_loss1 = win_loss > 0

In [238]:
maps = {True: 1, False: -1.5}

In [239]:
win_loss1 = win_loss1.map(maps)

In [240]:
win_loss1_by_account = win_loss1.groupby(level=0)

In [241]:
win_loss2 = win_loss1_by_account.sum()

In [242]:
win_loss2 = win_loss2.loc[submit.ACCOUNT_ID.tolist()]

In [243]:
win_loss2.describe()

count    5712.000000
mean       -1.302433
std         4.937498
min       -22.000000
25%        -4.000000
50%        -1.000000
75%         1.500000
max        19.000000
Name: correct_profit_loss, dtype: float64

In [230]:
win_loss2 = win_loss2.reset_index()

In [231]:
win_loss2.shape

(7374, 2)

In [232]:
win_loss2.columns = ['Account_ID', 'Prediction']

In [233]:
win_loss2 = win_loss2.fillna(-10)

In [234]:
win_loss2.Prediction.min()

-20.399999999999999

In [235]:
win_loss2.to_csv('four_team_count_games_win1_loss1_4_fill_minus10.csv', index=False)

In [236]:
win_loss2.isnull().any()

Account_ID    False
Prediction    False
dtype: bool

In [32]:
four_team_pre_dat_by_account_bet = four_team_pre_dat.groupby(['ACCOUNT_ID', 'BET_ID'])

In [33]:
win_loss = four_team_pre_dat_by_account_bet['correct_profit_loss'].sum()

In [34]:
win_loss1 = win_loss > 0

In [35]:
maps = {True: 1, False: -1}

In [36]:
win_loss1 = win_loss1.map(maps)

In [37]:
win_loss1_by_account = win_loss1.groupby(level=0)

In [38]:
win_loss2 = win_loss1_by_account.sum()

In [39]:
win_loss2 = win_loss2.loc[submit.ACCOUNT_ID.tolist()]

In [40]:
win_loss2 = win_loss2.reset_index()

In [41]:
win_loss2.columns = ['Account_ID', 'Prediction']

In [42]:
win_loss2.Prediction.describe()

count    6126.000000
mean       -4.839210
std        34.016698
min      -905.000000
25%        -9.000000
50%        -1.000000
75%         3.000000
max       960.000000
Name: Prediction, dtype: float64

In [43]:
win_loss2 = win_loss2.fillna(-5)

In [44]:
win_loss2.to_csv('four_team_count_bid_fill_minus5.csv', index=False)

In [45]:
win_loss2.isnull().any()

Account_ID    False
Prediction    False
dtype: bool

In [178]:
pre_qu_dat = four_team_pre_dat[~four_team_pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]

In [179]:
qu_dat = four_team_pre_dat[four_team_pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]

In [180]:
qu_is_profit = (qu_dat.groupby('ACCOUNT_ID')['correct_profit_loss'].sum() > 0).map({True: 1, False: 0})

In [181]:
qu_is_profit.shape

(4925,)

In [184]:
pre_qu_dat_by_account_bet = pre_qu_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [185]:
win_loss = pre_qu_dat_by_account_bet['correct_profit_loss'].sum()

In [186]:
best_loss = 0
best_default_ind = 0
best_auc = 0
for loss in np.linspace(-.5, -2.5, 21):
    print(loss)
    win_loss1 = win_loss > 0
    maps = {True: 1, False: loss}
    win_loss1 = win_loss1.map(maps)
    win_loss1_by_account = win_loss1.groupby(level=0)
    win_loss2 = win_loss1_by_account.sum()
    win_loss2 = win_loss2.loc[qu_is_profit.index.tolist()]
    not_null = win_loss2.loc[win_loss2.notnull()]
    not_null.sort()
    pos = range(2315)
    for ind in pos:
        val = not_null.iloc[ind]
        win_loss2 = win_loss2.fillna(val)
        est_auc = roc_auc_score(qu_is_profit, win_loss2)
        if est_auc > best_auc:
            best_auc = est_auc
            best_loss = loss
            best_default_ind = ind / 4925

-0.5
-0.6
-0.7
-0.8
-0.9
-1.0
-1.1
-1.2
-1.3
-1.4
-1.5
-1.6
-1.7
-1.8
-1.9
-2.0
-2.1
-2.2
-2.3
-2.4
-2.5


In [188]:
best_auc

0.73316546762589929

In [189]:
best_loss

-1.3999999999999999

In [190]:
best_default_val * 6126

0.0

In [183]:
y_true = np.array([0, 1, 0, 1])
y_scores = np.array([0.1, 0.35, 0.4, 0.8]) * 200
roc_auc_score(y_true, y_scores)

0.75

In [244]:
submit.ACCOUNT_ID.nunique()

7374

In [249]:
four_team_pre_dat.loc[four_team_pre_dat.ACCOUNT_ID.isin(submit.ACCOUNT_ID), 'ACCOUNT_ID'].nunique()

6126

In [250]:
pre_dat.ACCOUNT_ID.nunique()

6373

In [259]:
semi = pd.read_csv('semi_and_final_features.csv')
semi.columns = ['Account_ID', 'EVENT_ID', 'TRANSACTION_COUNT', 'STATUS_ID',
       'INPLAY_BET', 'AVG_BET_SIZE', 'MAX_BET_SIZE', 'MIN_BET_SIZE',
       'STDEV_BET_SIZE'] 

In [260]:
semi.Account_ID.nunique()

13087

In [282]:
new_ones = submit.loc[~submit.ACCOUNT_ID.isin(pre_dat.ACCOUNT_ID).values, 'ACCOUNT_ID'].tolist()

In [283]:
len(new_ones)

1001

In [284]:
new_dat = semi.loc[semi.Account_ID.isin(new_ones)]

In [288]:
new_dat.loc[:, 'total'] = new_dat.TRANSACTION_COUNT * new_dat.AVG_BET_SIZE

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [291]:
new_dat.groupby('Account_ID')['total'].sum().describe()

count        1001.000000
mean        48275.862071
std        556814.310914
min           200.470000
25%           400.000000
50%           822.000000
75%          2754.700790
max      13027739.484995
Name: total, dtype: float64

In [292]:
new_dat.loc[(new_dat.STATUS_ID == 'S').values, :].groupby('Account_ID')['total'].sum().describe()

count       1001.000000
mean       24020.470176
std       224208.247112
min          200.465336
25%          359.000000
50%          703.578600
75%         2158.539417
max      5387229.520910
Name: total, dtype: float64

In [293]:
new_dat.groupby('Account_ID')['total'].sum().idxmax()

1001423

In [294]:
new_dat.loc[(new_dat.Account_ID == 1001423).values, :]

,Account_ID,EVENT_ID,TRANSACTION_COUNT,STATUS_ID,INPLAY_BET,AVG_BET_SIZE,MAX_BET_SIZE,MIN_BET_SIZE,STDEV_BET_SIZE,total
2405,1001423,101187238,81,C,N,23719.081387,50584.006323,25.380522,18703.387660,1921245.592329
2406,1001423,101187238,81,C,N,19728.424149,50584.006323,63.223677,17067.077330,1598002.356084
2407,1001423,101187238,25,S,N,1011.680506,6368.860719,2.788443,1432.701485,25292.012646
2408,1001423,101187238,3,S,N,6744.535441,12646.006323,1264.606323,5702.397705,20233.606323
2409,1001423,101187238,724,S,Y,1703.450025,47220.233553,0.189690,4350.145533,1233297.817779
2410,1001423,101187238,586,S,Y,2009.760694,50584.006323,0.815667,5235.281885,1177719.766950
2411,1001423,101187943,53,C,N,4786.386243,14986.594618,62.867046,3812.277463,253678.470884
2412,1001423,101187943,46,C,N,5674.078321,25149.327046,125.752954,5210.607010,261007.602776
2413,1001423,101187943,358,S,Y,884.483859,16353.976446,0.132034,1973.124321,316645.221416
2414,1001423,101187943,205,S,Y,1348.456341,25149.327046,1.131720,2585.141014,276433.549960


In [295]:
new_dat.Account_ID.nunique()

1001

In [296]:
new_dat.loc[(new_dat.STATUS_ID == 'S').values, 'Account_ID'].nunique()

1001

In [298]:
new_dat.loc[(new_dat.STATUS_ID == 'S').values, :].groupby('Account_ID')['EVENT_ID'].nunique().value_counts()

1    610
2    280
3    111
dtype: int64

In [305]:
semi.loc[(semi.STATUS_ID == 'S').values].MAX_BET_SIZE.idxmax()

14875

In [308]:
semi.loc[(semi.STATUS_ID == 'S') & (semi.MAX_BET_SIZE > 250000)]

,Account_ID,EVENT_ID,TRANSACTION_COUNT,STATUS_ID,INPLAY_BET,AVG_BET_SIZE,MAX_BET_SIZE,MIN_BET_SIZE,STDEV_BET_SIZE
14875,1006014,101187238,83,S,Y,4614.778327,255509.159483,3.819862,28317.715663
23698,1009080,101187238,1584,S,Y,2277.857067,255509.159483,2.555092,8871.980495


In [312]:
win_loss2.loc[1009080]

1.0

In [306]:
semi.iloc[14875]

Account_ID             1006014
EVENT_ID             101187238
TRANSACTION_COUNT           83
STATUS_ID                    S
INPLAY_BET                   Y
AVG_BET_SIZE          4614.778
MAX_BET_SIZE          255509.2
MIN_BET_SIZE          3.819862
STDEV_BET_SIZE        28317.72
Name: 14875, dtype: object

In [307]:
semi.loc[14875]

Account_ID             1006014
EVENT_ID             101187238
TRANSACTION_COUNT           83
STATUS_ID                    S
INPLAY_BET                   Y
AVG_BET_SIZE          4614.778
MAX_BET_SIZE          255509.2
MIN_BET_SIZE          3.819862
STDEV_BET_SIZE        28317.72
Name: 14875, dtype: object

ACCOUNT_ID
1004240    2.0
1009742    5.5
1003908    2.0
1020888    NaN
1010074   -1.5
Name: correct_profit_loss, dtype: float64

In [315]:
y_true = np.array([0, 0, 0, 1])
y_scores = np.array([0, 0, 1, 1])
roc_auc_score(y_true, y_scores)

0.83333333333333337